In [1]:
import os
import shelve

import warnings
warnings.filterwarnings("ignore")

import torch
import numpy as np
from models.rgcn import RGCN

from prolog.prolog_generation import PrologData

from data.ag.action_genome import AG, AGViewer

%load_ext autoreload

pygame 2.6.1 (SDL 2.30.7, Python 3.9.20)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
%autoreload
# Load dataset
root = '/data/Datasets/ag/'
ag = AG(root, no_img=True)

In [3]:
# Load model
#model = RGCN(ag.num_nodes, ag.num_rels, ag.num_classes, ag.num_bases, ag.num_hidden, ag.num_layers, ag.dropout, ag.use_self_loop)
model = None

In [4]:
%autoreload
# Load prolog data
pd = PrologData('prolog', 'ag', ag, ag.object_classes, ag.relationship_classes, ag.verb_classes, 
                model=None, subset_file='data/ag/subset_shelve')

pd.write_bk()
pd.init_general_bias()

In [5]:
%autoreload
#need the verb count
with shelve.open('data/ag/subset_shelve') as subset_dict:
    agview = AGViewer(ag, subset_dict=subset_dict)
    total_valid, _, verb_freq, _ = agview.analyze_vocab_frequencies()


In [6]:
def exp_curve(b,x):
    return 1-np.exp(-b*x)

for verb_idx, verb_name in enumerate(ag.verb_classes):

    freq = verb_freq[verb_idx]
    ratio = freq/total_valid
    
    #keeps negatives according to the frequency of the verb
    pd.write_verb(verb_name, keep_prob=exp_curve(4, ratio)) 